In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [4]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
            'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch', 'EnclosedPorch']
for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')
print(num_att)

['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']


In [5]:
train.info()
# train.select_dtypes(include=['object'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
cat_att = ['BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Foundation', 'PavedDrive', 'SaleCondition']


In [7]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(median, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(median, inplace=True)

In [8]:
train[num_att].info()
test[num_att].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 24 columns):
LotFrontage     1460 non-null float64
LotArea         1460 non-null int64
OverallQual     1460 non-null int64
YearBuilt       1460 non-null int64
YearRemodAdd    1460 non-null int64
MasVnrArea      1460 non-null float64
BsmtFinSF1      1460 non-null int64
BsmtUnfSF       1460 non-null int64
TotalBsmtSF     1460 non-null int64
1stFlrSF        1460 non-null int64
2ndFlrSF        1460 non-null int64
GrLivArea       1460 non-null int64
BsmtFullBath    1460 non-null int64
FullBath        1460 non-null int64
HalfBath        1460 non-null int64
BedroomAbvGr    1460 non-null int64
KitchenAbvGr    1460 non-null int64
TotRmsAbvGrd    1460 non-null int64
Fireplaces      1460 non-null int64
GarageYrBlt     1460 non-null float64
GarageCars      1460 non-null int64
GarageArea      1460 non-null int64
WoodDeckSF      1460 non-null int64
OpenPorchSF     1460 non-null int64
dtypes: float64(3), int

In [9]:
train[cat_att].info()
test[cat_att].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 7 columns):
BldgType         1460 non-null object
Heating          1460 non-null object
HeatingQC        1460 non-null object
CentralAir       1460 non-null object
Foundation       1460 non-null object
PavedDrive       1460 non-null object
SaleCondition    1460 non-null object
dtypes: object(7)
memory usage: 80.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 7 columns):
BldgType         1459 non-null object
Heating          1459 non-null object
HeatingQC        1459 non-null object
CentralAir       1459 non-null object
Foundation       1459 non-null object
PavedDrive       1459 non-null object
SaleCondition    1459 non-null object
dtypes: object(7)
memory usage: 79.9+ KB


In [10]:
train_y = train["SalePrice"].copy()
# train = train.drop("SalePrice", axis=1)
# train = train.drop("Id", axis=1)
# test = test.drop("Id", axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [11]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 31 columns):
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-null int64
WoodDeckSF       1460 non-null int64
OpenPorchSF      1460 non-null int64

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [13]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 24)
(1459, 24)
(1460, 57)
(1459, 57)


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=200, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)
forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

10970.365755172124

In [15]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [25726.88970851 27336.58402455 22690.60335738 39915.86378968
 33111.70643356 25458.27732202 24632.46417221 24363.73575356
 37923.1628325  26848.83826976]
mean: 28800.81256637273
std: 5712.430236226304
count       10.000000
mean     28800.812566
std       6021.430174
min      22690.603357
25%      24838.917460
50%      26287.863989
75%      31667.925831
max      39915.863790
dtype: float64


In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [10, 30, 100, 200, 300], 'max_features': [4, 8, 16, 32, 57]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [10, 30, 100, 200, 300], 'max_features': [4, 8, 16, 32, 57]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_full_prepared, train_y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [4, 8, 16, 32, 

In [23]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 16, 'n_estimators': 200}

In [24]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=16, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [25]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

33080.462095284194 {'max_features': 4, 'n_estimators': 10}
31024.62165692982 {'max_features': 4, 'n_estimators': 30}
30398.10517242756 {'max_features': 4, 'n_estimators': 100}
30285.636464306244 {'max_features': 4, 'n_estimators': 200}
30294.0903876626 {'max_features': 4, 'n_estimators': 300}
31804.223410465493 {'max_features': 8, 'n_estimators': 10}
29497.082955718273 {'max_features': 8, 'n_estimators': 30}
29286.635514817317 {'max_features': 8, 'n_estimators': 100}
29454.11806347448 {'max_features': 8, 'n_estimators': 200}
29271.008519810024 {'max_features': 8, 'n_estimators': 300}
31555.071510174494 {'max_features': 16, 'n_estimators': 10}
30196.55462783468 {'max_features': 16, 'n_estimators': 30}
29503.085144678003 {'max_features': 16, 'n_estimators': 100}
29158.63520544974 {'max_features': 16, 'n_estimators': 200}
29039.094626022114 {'max_features': 16, 'n_estimators': 300}
32177.202726305575 {'max_features': 32, 'n_estimators': 10}
30384.00346748867 {'max_features': 32, 'n_estima

In [26]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([1.04513797e-02, 2.30009589e-02, 2.29423464e-01, 7.03172629e-02,
       2.24431348e-02, 1.06085227e-02, 3.38199235e-02, 9.59472002e-03,
       7.84437097e-02, 5.75066322e-02, 4.19219023e-02, 1.43835779e-01,
       3.06039617e-03, 1.95225029e-02, 5.85999344e-03, 5.13244475e-03,
       2.27851733e-03, 1.68589940e-02, 1.22616229e-02, 2.35250485e-02,
       8.60441464e-02, 5.06794271e-02, 5.72760464e-03, 8.60392936e-03,
       1.52584922e-03, 1.05497873e-04, 4.52039024e-04, 9.73719399e-05,
       2.66174630e-04, 2.70393205e-06, 2.89267792e-04, 1.58566166e-04,
       2.30763124e-05, 5.97017650e-05, 1.82455492e-06, 2.14502877e-03,
       2.80904686e-04, 5.72026294e-04, 1.37822482e-06, 7.70366872e-04,
       2.11426901e-03, 2.32425762e-03, 4.75128677e-04, 1.19787703e-03,
       1.05233209e-02, 1.02019676e-04, 6.12112514e-05, 1.71094414e-05,
       4.51793303e-04, 1.47756850e-04, 4.74220427e-04, 1.22091083e-03,
       1.33172786e-05, 3.03954589e-04, 3.29952094e-04, 1.23491922e-03,
      

In [27]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.22942346380745568, 'OverallQual'),
 (0.14383577935147474, 'GrLivArea'),
 (0.08604414639517392, 'GarageCars'),
 (0.07844370968270444, 'TotalBsmtSF'),
 (0.07031726289574812, 'YearBuilt'),
 (0.057506632240132145, '1stFlrSF'),
 (0.05067942712207919, 'GarageArea'),
 (0.04192190232546536, '2ndFlrSF'),
 (0.0338199235334144, 'BsmtFinSF1'),
 (0.02352504845532305, 'GarageYrBlt'),
 (0.023000958886941, 'LotArea'),
 (0.022443134775635806, 'YearRemodAdd'),
 (0.019522502879869914, 'FullBath'),
 (0.016858993982226577, 'TotRmsAbvGrd'),
 (0.012261622902419589, 'Fireplaces'),
 (0.010608522651490208, 'MasVnrArea'),
 (0.01045137966475273, 'LotFrontage'),
 (0.00959472002017236, 'BsmtUnfSF'),
 (0.008603929359858565, 'OpenPorchSF'),
 (0.005859993441758663, 'HalfBath'),
 (0.005727604641504114, 'WoodDeckSF'),
 (0.005132444746263965, 'BedroomAbvGr'),
 (0.003060396173561667, 'BsmtFullBath'),
 (0.002278517326570574, 'KitchenAbvGr'),
 (0.0015258492202440401, '1Fam'),
 (0.00045203902378664664, 'Duplex'),
 (0.000

In [28]:
y_pred = grid_search.predict(test_full_prepared)
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred})

In [29]:
sub.to_csv("data/submission_script10_rf_gridsearch_less_att.csv", index=False)

In [30]:
sub.shape

(1459, 2)